In [3]:
import numpy as np
import pandas as pd
import re
import string

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def count_remover(text,threshold=4):
    if len(text.split())<threshold:
        return pd.NaT
    else:
        return text

data=pd.read_json('tweets.json' ,lines=True)
df=data[["Text","CreatedAt"]].rename(columns={"Text":"content","CreatedAt":"time"})
df['content'] = df['content'].str.replace(r'@\w+', '')
df['content'] = df['content'].apply(lambda x: re.sub(r"http\S+", "", x))
df['content'] = df['content'].apply(lambda x: remove_punct(x))
df['content'] = df['content'].apply(lambda x: count_remover(x))
df=df.dropna()
documents_EM_tweets=df.content.tolist()
#documents_EMsTweets=[document for document in documents if len(document.split())>10]

In [4]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups
import swifter

stop_words = list(set(stopwords.words('english')))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[{}0-9]'.format(string.punctuation), ' ', text)
    text=re.sub(r'[^A-Za-z0-9 ]+', ' ', text)
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = [WordNetLemmatizer().lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df=pd.DataFrame({"content":newsgroups["data"]})

df["content"]=df["content"].swifter.apply(lambda x: preprocess_text(x))
df['content_length'] = df['content'].str.len()

df = df[df['content_length'] > 100]
df = df[df['content_length'] < 2000]

df=df[["content"]].reset_index(drop=True).reset_index().rename(columns={"index":"id"})
documents_20newsgroup=df.content.to_list()

Pandas Apply:   0%|          | 0/18846 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
import time
import os
import pickle
import numpy as np

ntm_results_path = "/Users/hamed/PycharmProjects/CTC/NTM/results/"
coherence_result = "/Users/hamed/PycharmProjects/CTC/ctc_result.txt"

with open(coherence_result, "w") as f:
    cpmi_result=[]
    for documents in [(documents_EM_tweets,"tweets"),(documents_20newsgroup,"20newsgroup")]:
        sentences=[sentence.strip().lower() for sentence in documents[0]]    #lowering the words in the dataset
        sentences  = ["".join([char for char in text if char not in string.punctuation]) for text in sentences] #remove punctuation
        sentences = [sentence.split(' ') for sentence in sentences] #tokenizing the dataset
        sentences=[[token for token in sentence if len(token)>0] for sentence in sentences]
        sentences=[sentence for sentence in sentences if len(sentence) > 5] #filtering the dataset
        sentences = [sentence[i:i+20] for sentence in sentences for i in range(0, len(sentence), 20) ]
        sentences=[sentence for sentence in sentences if len(sentence) > 5]

        model_counter=0
        for topic_model in os.listdir(ntm_results_path):
            if documents[1] in topic_model.split("_") and 'topics' in topic_model.split("_"):
                model_counter+=1

        cpmi_matrix=np.zeros((len(sentences),model_counter))
        c=1
        for i,sentence in enumerate(tqdm(sentences)):


            if len(sentence)>5:
                try:

                    outs = txt_to_pmi.get_cpmi(MODEL, [sentence], verbose=False)
                    for j,filename in enumerate(os.listdir(ntm_results_path)):
                        sentence_topic_score=0
                        name=filename.split("_")
                        if ('topics' in name) and (documents[1] in name):
                            with open("/Users/hamed/PycharmProjects/CTC/NTM/results/"+str(filename), "rb") as fp:
                                topics = pickle.load(fp)
                            for topic in topics:
                                topic_score=0
                                words=list(set(topic) & set(sentence))
                                if len(words)>1 :
                                    for word1 in words:
                                        w1=sentence.index(word1)
                                        for word2 in words:
                                            w2=sentence.index(word2)
                                            topic_score+=outs[0]["0"][w1][w2]/2
                                sentence_topic_score+=topic_score/len(topic)
                        cpmi_matrix[i][j]=sentence_topic_score/len(topics)
                        c+=1
                except:

                    continue
        for j,filename in enumerate(os.listdir(ntm_results_path)):
            f.write(f"{filename}, {documents[1]} ,ctc, {np.sum(cpmi_matrix[:][j])}\n")
        cpmi_result.append(cpmi_matrix)
    f.write("The experiment is successfully done!")

In [3]:
def cpmi_coherence(topics,documents,batch_size=50,min_batch_lenghth=5):
    sentences=[sentence.strip().lower() for sentence in documents]    #lowering the words in the dataset
    sentences  = ["".join([char for char in text if char not in string.punctuation]) for text in sentences] #remove punctuation
    sentences = [sentence.split(' ') for sentence in sentences] #tokenizing the dataset
    sentences=[[token for token in sentence if len(token)>0] for sentence in sentences]
    sentences=[sentence for sentence in sentences if len(sentence) > min_batch_lenghth] #filtering the dataset
    #batching the dataset
    print(len(sentences))
    sentences = [sentence[i:i+batch_size] for sentence in sentences for i in range(0, len(sentence), batch_size) ]
    print(len(sentences))
    #calculating CPMI for each batch
    sentence_counter=0
    coherence=0
    for i,sentence in enumerate(tqdm(sentences)):
        sentence_score=0
        try:
            outs = txt_to_pmi.get_cpmi(MODEL, [sentence], verbose=False)
            sentence_counter+=1
            for topic in topics:
                topic_score=0
                words=list(set(topic) & set(sentence))
                if len(words)>1 :
                    for word1 in words:
                        w1=sentence.index(word1)
                        for word2 in words:
                            w2=sentence.index(word2)
                            topic_score+=outs[0]["0"][w1][w2]/2
                sentence_score+=topic_score/len(topic)
        except:
            print("ERORR in CPMI")
            continue
        coherence+=sentence_score

    return coherence/sentence_counter

In [5]:
import torch
import txt_to_pmi
DEVICE = torch.device('mps')
MODEL = txt_to_pmi.languagemodel.BERT(DEVICE, 'bert-base-cased', 32)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentences=[sentence.strip().lower() for sentence in documents_20newsgroup]
sentences  = ["".join([char for char in text if char not in string.punctuation]) for text in sentences]
sentences = [sentence.split(' ') for sentence in sentences]
sentences=[[token for token in sentence if len(token)>0] for sentence in sentences]
sentences=[sentence for sentence in sentences if len(sentence) > 5]
sentences = [sentence[i :i+20] for sentence in sentences for i in range(0, len(sentence), 20) ]
sentences=[sentence for sentence in sentences if len(sentence) > 5]
outs = txt_to_pmi.get_cpmi(MODEL, sentences, verbose=False)

  0%|          | 8/53363 [00:17<37:27:24,  2.53s/it]

In [ ]:

with open("sentences_20", "wb") as fp:   #Pickling
  pickle.dump(sentences, fp)

with open("out_20", "wb") as fp:   #Pickling
  pickle.dump(outs, fp)

In [ ]:
from tqdm import tqdm
import pickle
with open("/Users/hamed/PycharmProjects/CTC/NTM/results/etm_20newsgroup_20_topics", "rb") as fp:
    topics = pickle.load(fp)
cpmi_coherence(topics,documents_20newsgroup,batch_size=15,min_batch_lenghth=5)

14624
74056


 12%|█▏        | 8891/74056 [2:24:36<18:55:06,  1.05s/it]

ERORR in CPMI
